In [2]:
import pandas as pd

In [245]:
feat_import = pd.read_csv('data/output/feature_importance.csv')
feat_import

,imp,feature
0,2.914117e-02,220045
1,7.660621e-04,220046
2,4.910010e-05,220047
3,0.000000e+00,220074
4,1.553653e-03,220179
...,...,...
1442,3.473629e-03,ethnicity_6
1443,0.000000e+00,ethnicity_7
1444,1.307571e-03,insurance_0
1445,1.158232e-08,insurance_1


In [246]:
# contains metadata for lab ids
d_labitems = pd.read_csv("mimiciv/1.0/icu/d_items.csv.gz")
d_labitems

,itemid,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
0,220003,ICU Admission date,ICU Admission date,datetimeevents,ADT,NaN,Date and time,NaN,NaN
1,220045,Heart Rate,HR,chartevents,Routine Vital Signs,bpm,Numeric,NaN,NaN
2,220046,Heart rate Alarm - High,HR Alarm - High,chartevents,Alarms,bpm,Numeric,NaN,NaN
3,220047,Heart Rate Alarm - Low,HR Alarm - Low,chartevents,Alarms,bpm,Numeric,NaN,NaN
4,220048,Heart Rhythm,Heart Rhythm,chartevents,Routine Vital Signs,NaN,Text,NaN,NaN
...,...,...,...,...,...,...,...,...,...
3856,229355,Absolute Neutrophil Count,Absolute Neutrophil Count,chartevents,Labs,NaN,Numeric,NaN,NaN
3857,229453,Exam-GI/GU,Exam-GI/GU,chartevents,MD Progress Note,NaN,Text,NaN,NaN
3858,229604,Therapeutic Bed,Therapeutic Bed,chartevents,Treatments,NaN,Text,NaN,NaN
3859,229709,Angiotensin II (Giapreza),Angiotensin II (Giapreza),inputevents,Medications,mg,Solution,NaN,NaN


In [247]:
feat_mask = feat_import.feature.apply(lambda x: x[0].isdigit())
feat_import_itemid = feat_import.feature[feat_mask].astype(int)
feat_import_itemid

0      220045
1      220046
2      220047
3      220074
4      220179
        ...  
403    224318
404    226170
405    224728
406    228628
407    228731
Name: feature, Length: 408, dtype: int64

In [248]:
flowlab_metadata = d_labitems[['itemid', 'label']]
flowlab_metadata['itemid'] = flowlab_metadata.itemid.astype('str')
flowlab_metadata = flowlab_metadata.rename({"label": "meta", "itemid": "feature"}, axis=1)
flowlab_metadata['feature_type'] = 'lab/flowsheet'
flowlab_metadata

/var/folders/wd/p54fgc1d6nd09_tvl49r2flc0000gp/T/ipykernel_20921/3249835777.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flowlab_metadata['itemid'] = flowlab_metadata.itemid.astype('str')


,feature,meta,feature_type
0,220003,ICU Admission date,lab/flowsheet
1,220045,Heart Rate,lab/flowsheet
2,220046,Heart rate Alarm - High,lab/flowsheet
3,220047,Heart Rate Alarm - Low,lab/flowsheet
4,220048,Heart Rhythm,lab/flowsheet
...,...,...,...
3856,229355,Absolute Neutrophil Count,lab/flowsheet
3857,229453,Exam-GI/GU,lab/flowsheet
3858,229604,Therapeutic Bed,lab/flowsheet
3859,229709,Angiotensin II (Giapreza),lab/flowsheet


In [249]:
flowlab_metadata[flowlab_metadata.meta.str.lower().str.contains('sofa')] #sofa

,feature,meta,feature_type
1747,227428,SOFA Score,lab/flowsheet


In [250]:
# get mapping for ICD10 codes. this isnt a perfect csv but it will do for now
icd10_mapping = pd.read_csv('utils/mappings/icd10_codes.csv', names=[
    'ICD10_parent',
    'ICD10_sub',
    'ICD10_full',
    'diagnosis_name',
    'diagnosis_name_more',
    'parent_diagnosis_name',
])
icd10_mapping = icd10_mapping[['ICD10_parent', 'parent_diagnosis_name']].drop_duplicates().reset_index(drop=True)
icd10_mapping = icd10_mapping.rename({"parent_diagnosis_name": "meta", "ICD10_parent": "feature"}, axis=1)
icd10_mapping['feature_type'] = 'diagnosis'
icd10_mapping

,feature,meta,feature_type
0,A00,Cholera,diagnosis
1,A010,Typhoid fever,diagnosis
2,A011,Paratyphoid fever A,diagnosis
3,A012,Paratyphoid fever B,diagnosis
4,A013,Paratyphoid fever C,diagnosis
...,...,...,...
19922,Z99,"Dependence on enabling machines and devices, n...",diagnosis
19923,Z991,Dependence on respirator,diagnosis
19924,Z992,Dependence on renal dialysis,diagnosis
19925,Z993,Dependence on wheelchair,diagnosis


In [251]:
dx_feat = feat_import.merge(icd10_mapping, on='feature')
flowlab_feat = feat_import.merge(flowlab_metadata, on='feature')
metadata = pd.concat([flowlab_feat, dx_feat])[['feature', 'meta', 'feature_type']]
metadata.to_csv('data/output/metadata.csv', index=False)
metadata

,feature,meta,feature_type
0,220045,Heart Rate,lab/flowsheet
1,220046,Heart rate Alarm - High,lab/flowsheet
2,220047,Heart Rate Alarm - Low,lab/flowsheet
3,220074,Central Venous Pressure,lab/flowsheet
4,220179,Non Invasive Blood Pressure systolic,lab/flowsheet
...,...,...,...
649,L85,Other epidermal thickening,diagnosis
650,R17,Unspecified jaundice,diagnosis
651,Q99,"Other chromosome abnormalities, not elsewhere ...",diagnosis
652,Q02,Microcephaly,diagnosis


In [252]:
# stuff that's still unmatched
feat_import[~feat_import.feature.isin(dx_feat.feature.tolist() + flowlab_feat.feature.tolist())]

,imp,feature
415,0.000000e+00,Z85
416,0.000000e+00,R09
421,2.651000e-03,E11
422,0.000000e+00,C78
426,6.802825e-03,T78
...,...,...
1442,3.473629e-03,ethnicity_6
1443,0.000000e+00,ethnicity_7
1444,1.307571e-03,insurance_0
1445,1.158232e-08,insurance_1


In [253]:
feat_import = feat_import.merge(metadata, on="feature")

In [254]:
flowlab_metadata[flowlab_metadata.meta.str.lower().str.contains('peptide')]

,feature,meta,feature_type
1079,225622,ZBrain Natiuretic Peptide (BNP),lab/flowsheet
1757,227446,Brain Natiuretic Peptide (BNP),lab/flowsheet


In [255]:
feat_import.sort_values('imp', ascending=False).iloc[:20]

,imp,feature,meta,feature_type
34,0.031678,227466,PTT,lab/flowsheet
0,0.029141,220045,Heart Rate,lab/flowsheet
421,0.028256,N17,Acute kidney failure,diagnosis
30,0.025861,227442,Potassium (serum),lab/flowsheet
174,0.023846,224751,Temporary Pacemaker Rate,lab/flowsheet
441,0.022541,K92,Other diseases of digestive system,diagnosis
9,0.022290,220277,O2 saturation pulseoxymetry,lab/flowsheet
19,0.021480,223761,Temperature Fahrenheit,lab/flowsheet
81,0.016098,225690,Total Bilirubin,lab/flowsheet
102,0.014559,225643,Differential-Neuts,lab/flowsheet


In [256]:
feat_import[feat_import.feature == '227446']

,imp,feature,meta,feature_type
278,0.005906,227446,Brain Natiuretic Peptide (BNP),lab/flowsheet


In [257]:
a = pd.read_csv('data/csv/32773003/dynamic.csv', header=1)
print(a.shape)
a['227428']

(3, 408)


KeyError: '227428'

In [7]:
pd.read_csv("mimiciv/1.0/hosp/diagnoses_icd.csv.gz").sort_values(['hadm_id', 'seq_num'])

,subject_id,hadm_id,seq_num,icd_code,icd_version
2019897,10467237,20000019,1,0389,9
2019893,10467237,20000019,2,59080,9
2019895,10467237,20000019,3,75310,9
2019896,10467237,20000019,4,5849,9
2019892,10467237,20000019,5,2761,9
...,...,...,...,...,...
4018657,16549335,29999928,7,Z8241,10
4018649,16549335,29999928,8,I959,10
4018654,16549335,29999928,9,I493,10
4018648,16549335,29999928,10,D500,10
